In [1]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [22]:
pip install tqdm rouge bert-score


Defaulting to user installation because normal site-packages is not writeable
  Using cached bert_score-0.3.13-py3-none-any.whl (61 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
#QG model from Hugging face
# from tqdm import tqdm
# from transformers import T5Config, T5ForConditionalGeneration, T5Tokenizer

# model_name = "allenai/t5-small-squad2-question-generation"
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name)

# def run_model(input_string, **generator_args):
#     input_ids = tokenizer.encode(input_string, return_tensors="pt")
#     res = model.generate(input_ids, **generator_args)
#     output = tokenizer.batch_decode(res, skip_special_tokens=True)
#     #print(output)
#     return output


In [5]:
!pip install pipeline

Defaulting to user installation because normal site-packages is not writeable


In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("anas-awadalla/bart-base-finetuned-squad-seq2seq")
model = AutoModelForSeq2SeqLM.from_pretrained("anas-awadalla/bart-base-finetuned-squad-seq2seq")
def run_model(input_string, **generator_args):
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    #print(output)
    return output


In [7]:
#for data_squad_v2
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable


In [8]:
from datasets import load_dataset
squads_v2 = load_dataset("squad_v2")

In [9]:
import torch
import datasets

train_data = squads_v2["train"]


In [10]:
#print(train_data[0])

In [11]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [12]:
#for metrics
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [13]:
from nltk.tokenize import word_tokenize

In [14]:
# from nltk.translate import meteor_score
# import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# reference = [nltk.word_tokenize(train_data[0]['question'])]
# hypothesis_array = run_model(train_data[0]['context'])
# final_hypothesis_array = []
# for item in hypothesis_array:
#     hypothesis = nltk.word_tokenize(item)
#     final_hypothesis_array.append(hypothesis)

# print(final_hypothesis_array)
# # Calculate METEOR score
# meteor = 0
# for item in final_hypothesis_array:
#     meteor = max(meteor_score.meteor_score(reference, item), meteor)

# print(f"METEOR Score: {meteor:.2f}")


In [15]:
import tqdm

In [18]:
# pip install rouge
# pip install bert-score
# !python -m pip install rouge

Defaulting to user installation because normal site-packages is not writeable


In [23]:
from rouge import Rouge

In [28]:
from nltk.translate import meteor_score
import nltk
from transformers import BertTokenizer
from bert_score import BERTScorer
from nltk.translate.bleu_score import corpus_bleu

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/asachd14/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/asachd14/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
rouge = Rouge()

# Define the reference once since it doesn't change

# Initialize a list to store the METEOR scores for each item
meteor_scores = []
corpus_scores = []
rouge_scores = []
bert_scores = []
train_data_list = []
# print(type(train_data))
# sample = train_data.sample(frac=0.05, random_state=42) 
sample = train_data.select(range(int(len(train_data) * 0.005)))
print(len(sample))
for i in range(len(sample)):
    train_data_list.append(sample[i])

tokenizer_bert_score = BertTokenizer.from_pretrained("bert-base-uncased")
scorer = BERTScorer(lang="en", rescale_with_baseline=True)

for item in train_data_list:
    #print('ANswers are',item['answers'])
    reference = nltk.word_tokenize(item['question'])
    hypothesis_array = run_model(f"Context: {item['context']} [SEP] Answer: {' '.join(item['answers'])}")
    final_hypothesis_array = [nltk.word_tokenize(h) for h in hypothesis_array]

    # print("Reference:", reference)
    # print("Hyopthesis:", final_hypothesis_array[0])
    
    # Calculate METEOR score for each hypothesis
    # print("METEOR")
    meteor = meteor_score.meteor_score([reference], final_hypothesis_array[0])
    meteor_scores.append(meteor)
    # print(meteor)

    # Calculate BLEU score for each hypothesis
    # print("BLEU")
    corpus_bleud = corpus_bleu([reference], final_hypothesis_array)
    corpus_scores.append(corpus_bleud)
    # print(corpus_bleud)

    # Calculate ROUGE scores
    # print("ROUGE")
    reference_rouge = " ".join(nltk.word_tokenize(item['question']))  # Convert the list to a string
    hypothesis_rouge = " ".join(run_model(f"Context: {item['context']} [SEP] Answer: {' '.join(item['answers'])}"))
    item_scores_rouge = rouge.get_scores(reference_rouge, hypothesis_rouge)
    rouge_scores.append(item_scores_rouge)
    # print(item_scores_rouge)

    # Calculate BERTscore scores
    # print("BERTscore")
    reference_bert = " ".join(tokenizer.tokenize(item['question']))
    hypothesis_bert = " ".join(run_model(f"Context: {item['context']} [SEP] Answer: {' '.join(item['answers'])}"))
    P, R, F1 = scorer.score([reference_bert],[hypothesis_bert])

    item_scores_bert = {
        "Precision": P,
        "Recall": R,
        "F1 Score": F1,
    }
    bert_scores.append(item_scores_bert)
    # print(item_scores_bert)
    # print(item_scores_bert['Precision'])
    # print(item_scores_bert['Recall'])
    # print(item_scores_bert['F1 Score'])

    

average_meteor = sum(meteor_scores) / len(meteor_scores)
print(f"Average METEOR Score: {average_meteor:.2f}")

average_corpus = sum(corpus_scores) / len(corpus_scores)
print(f"Average corpus-BLEU Score: {average_corpus}")  # Fixed metric name

# average_rouge = sum(rouge_scores) / len(rouge_scores)
# print(f"Average Rouge Score: {average_rouge}")

651


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/asachd14/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
for i , itb in enumerate(bert_scores):
    print(f"BERTScore Scores for Item {i + 1}:")
    print("Precision:" ,itb['Precision'])
    print("Recall:" ,itb['Recall'])
    print("F1 Score:" ,itb['F1 Score'])
    print("")